In [1]:
# Data Cleaning
## data loading
from google.colab import drive
import pandas as pd
import requests
import io
import csv
import os
import altair as alt

# Print the current working directory
print("Current working directory:", os.getcwd())

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/cleaned_reduced_dataset.csv'

# Read the original CSV content
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Replace triple quotes with a single quote character
modified_content = content.replace('"""', '|')

# Create an in-memory file-like object with the modified content
data = io.StringIO(modified_content)

# Read the modified CSV content into a Pandas DataFrame
df = pd.read_csv(
    data,
    quoting=csv.QUOTE_MINIMAL,
    quotechar='|'
)

# Display the DataFrame
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True)
df = df.sort_values(by='DATE')

def count_words(text):
    return len(text.split())

# Apply the function to the 'TWEET' column
df['WORD_COUNT'] = df['TWEET'].apply(count_words)

# Display data
df.head()

Current working directory: /content
Mounted at /content/drive


,index,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY,MENTION,WORD_COUNT
0,0,RT @robertoglezcano: @amazon #Patents Show Fl...,Amazon,2017-01-31,823.4800,0.008379,0.014924,0.014924,-0.001263,3137196.0,13.447,16.992,1,0.0,@amazon,16
1171,1171,RT @StudentBunker: #Competition time! 💖 Simply...,ASOS,2017-01-31,5266.0000,-0.012533,0.008355,0.008355,-0.020699,342823.0,32.807,28.367,1,0.0,@ASOS,23
1170,1170,RT @RedCook36: @MaestroClaudio @aslmilf @bblin...,AT&T,2017-01-31,42.1600,-0.008065,-0.003558,-0.003558,-0.018975,25405353.0,14.379,15.465,1,0.0,@ATT,17
1169,1169,Nurse Infinity Charm Bracelet Jewelry - Nurse ...,Amazon,2017-01-31,823.4800,0.008379,0.014924,0.014924,-0.001263,3137196.0,13.447,16.992,1,0.0,@amazon,15
1168,1168,RT @Reuters: San Francisco sues Trump over san...,Reuters,2017-01-31,49.3887,-0.002677,0.003122,0.003122,0.004461,547029.0,9.665,9.094,1,0.0,@Reuters,12


In [2]:
# Calculates the mean values for certain columns grouped by the 'DATE' column
df_mean = df[['DATE', 'STOCK','1_DAY_RETURN',	'2_DAY_RETURN',	'3_DAY_RETURN','7_DAY_RETURN', 'VOLATILITY_10D','VOLATILITY_30D','TEXTBLOB_POLARITY']].drop_duplicates().groupby('DATE')[
    ['1_DAY_RETURN',	'2_DAY_RETURN',	'3_DAY_RETURN','7_DAY_RETURN', 'VOLATILITY_10D','VOLATILITY_30D','TEXTBLOB_POLARITY']].mean()
df_mean.columns = [f"{x}_mn" for x in df_mean.columns]
df_mean.head()

,1_DAY_RETURN_mn,2_DAY_RETURN_mn,3_DAY_RETURN_mn,7_DAY_RETURN_mn,VOLATILITY_10D_mn,VOLATILITY_30D_mn,TEXTBLOB_POLARITY_mn
DATE,,,,,,,
2017-01-31,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119
2017-02-01,-0.000937,0.005002,0.010789,0.004111,20.939843,17.462664,0.114120
2017-02-28,0.004920,0.004588,0.004588,0.010108,16.235928,19.377473,0.192934
2017-03-01,-0.009616,-0.005166,-0.005706,-0.005073,16.583284,19.145089,0.150476
2017-03-31,0.001078,-0.002087,-0.005136,-0.010717,14.600649,14.313712,0.167874


In [3]:
# Normalize the data by subtract mean
df1 = df.reset_index().merge(df_mean, on=['DATE'], how = 'right')
for col in ['1_DAY_RETURN',	'2_DAY_RETURN',	'3_DAY_RETURN','7_DAY_RETURN', 'VOLATILITY_10D','VOLATILITY_30D','TEXTBLOB_POLARITY']:
  df1[col] = df1[col]-df1[f'{col}_mn']
df1.head()

,index,index,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,...,TEXTBLOB_POLARITY,MENTION,WORD_COUNT,1_DAY_RETURN_mn,2_DAY_RETURN_mn,3_DAY_RETURN_mn,7_DAY_RETURN_mn,VOLATILITY_10D_mn,VOLATILITY_30D_mn,TEXTBLOB_POLARITY_mn
0,0,0,RT @robertoglezcano: @amazon #Patents Show Fl...,Amazon,2017-01-31,823.4800,0.002072,0.001633,0.001633,0.000260,...,-0.140119,@amazon,16,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119
1,1171,1171,RT @StudentBunker: #Competition time! 💖 Simply...,ASOS,2017-01-31,5266.0000,-0.018841,-0.004936,-0.004936,-0.019176,...,-0.140119,@ASOS,23,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119
2,1170,1170,RT @RedCook36: @MaestroClaudio @aslmilf @bblin...,AT&T,2017-01-31,42.1600,-0.014372,-0.016849,-0.016849,-0.017453,...,-0.140119,@ATT,17,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119
3,1169,1169,Nurse Infinity Charm Bracelet Jewelry - Nurse ...,Amazon,2017-01-31,823.4800,0.002072,0.001633,0.001633,0.000260,...,-0.140119,@amazon,15,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119
4,1168,1168,RT @Reuters: San Francisco sues Trump over san...,Reuters,2017-01-31,49.3887,-0.008984,-0.010169,-0.010169,0.005983,...,-0.140119,@Reuters,12,0.006307,0.013291,0.013291,-0.001523,21.590785,18.028871,0.140119


In [4]:
df_sampled = df.sample(n=1000, random_state=1)

In [5]:
# Explore the df volatility data
df['VOLATILITY_10D'].describe()

,VOLATILITY_10D
count,22295.000000
mean,18.374805
std,10.404547
min,3.064000
25%,11.148000
50%,16.195000
75%,23.420000
max,124.137000


In [6]:
# Explore the df1 volatility data
df1['VOLATILITY_10D_mn'].describe()

,VOLATILITY_10D_mn
count,22295.000000
mean,18.640267
std,3.235479
min,13.635617
25%,15.282732
50%,19.569225
75%,21.289172
max,26.007917


In [7]:
# Explore the df_sample volatility data
df_sampled['VOLATILITY_10D'].describe()

,VOLATILITY_10D
count,1000.000000
mean,17.890619
std,9.705955
min,4.034000
25%,10.600000
50%,15.921000
75%,23.337000
max,92.245000


In [8]:
# Calculate the tweet count for each stock by date.
df_tweet_counts = df_sampled.groupby(['STOCK', 'DATE']).size().reset_index(name='TWEET_COUNT')

# Merge the tweet counts back to the original dataframe
df_sampled = df_sampled.merge(df_tweet_counts, on=['STOCK', 'DATE'], how='left')

df_sampled.head()

,index,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY,MENTION,WORD_COUNT,TWEET_COUNT
0,8715,RT @FifthHarmony: Who's hanging with us in Mia...,AT&T,2017-05-31,38.5300,0.000519,-0.010641,-0.010641,-0.009862,24040423.0,9.834,14.120,-1,0.000000,@ATT,18,3
1,3199,Check out my listing on @eBay: https://t.co/DV...,eBay,2017-02-28,33.9000,0.011209,0.004720,0.004720,0.000590,8105294.0,15.920,21.975,-1,0.000000,@eBay,9,13
2,4418,I just checked in @McDonalds with #mPLACES to ...,McDonald's,2017-03-31,129.6100,-0.002237,-0.005941,-0.001234,-0.002083,3041677.0,4.766,6.792,1,0.000000,@McDonalds,17,11
3,6307,RT @Reuters: Pope Francis arrives in Cairo see...,Reuters,2017-04-28,50.0386,-0.035660,-0.052609,-0.055032,-0.061634,2448752.0,23.420,15.044,-1,0.000000,@Reuters,15,15
4,22029,RT @charliekirk11: Incredible video: @CBS does...,CBS,2017-12-30,59.0000,0.000000,0.004407,0.004915,0.019831,2334987.0,12.854,26.593,1,0.275397,@CBS,24,2


In [9]:
# Rank Stocks based on volatility(10 Days)

# Calculate mean 10-day volatility for each unique stock
df_volatility_mean = df_sampled.groupby('STOCK')['VOLATILITY_10D'].mean().reset_index()

# Sort the stocks by their 10-day volatility in descending order
df_volatility_mean = df_volatility_mean.sort_values(by='VOLATILITY_10D', ascending=False)

# Create the bar chart to show 10-day volatility
vol10_chart1 = alt.Chart(df_volatility_mean).mark_bar().encode(
    y=alt.Y('STOCK', sort='-x', title='Stock'),
    x=alt.X('VOLATILITY_10D', title='Mean 10-Day Volatility'),
    tooltip=['STOCK', 'VOLATILITY_10D']
).properties(
    title='Mean 10-Day Volatility of Unique Stocks',
    width=600,
    height=1200
).interactive()

# Display the chart
vol10_chart1

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
# Selection object to link the charts
selection = alt.selection_single(fields=['STOCK'], empty='none')

# Bar chart
vol10_chart1 = alt.Chart(df_volatility_mean).mark_bar().encode(
    y=alt.Y('STOCK', sort='-x', title='Stock'),
    x=alt.X('VOLATILITY_10D', title='Mean 10-Day Volatility'),
    tooltip=['STOCK', 'VOLATILITY_10D'],
    color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray'))
).properties(
    title='Mean 10-Day Volatility of Unique Stocks (Descending Order)',
    width=600,
    height=1200
).add_selection(
    selection
)

# Create the scatter plot to show the correlation between stock 10-day volatility and tweet counts
vol10_chart2 = alt.Chart(df_sampled).mark_point().encode(
    x=alt.X('TWEET_COUNT:Q', title='Tweet Count', scale=alt.Scale(zero=False)),
    y=alt.Y('VOLATILITY_10D:Q', title='10-Day Volatility', scale=alt.Scale(zero=False)),
    tooltip=['DATE', 'TWEET_COUNT', 'VOLATILITY_10D']
).transform_filter(
    selection
).properties(
    title='Correlation between Tweet Count and 10-Day Volatility for Selected Stock',
    width=800,
    height=500
)

# Add the trendline
trendline = vol10_chart2.transform_regression(
    'TWEET_COUNT', 'VOLATILITY_10D'
).mark_line(color='red')

# Combine the scatterplot and trendline
vol10_chart3 = vol10_chart2 + trendline

# Combine the charts vertically
combined_chart_vol10 = alt.hconcat(
    vol10_chart1,
    vol10_chart3
).resolve_scale(
    y='independent'
)

# Display the chart
combined_chart_vol10

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

In [11]:
# Exclude stocks where there is insufficient data for a meaningful correlation analysis. My definition is at least 2 days of data for a stock to consider it for correlation analysis.

# Filter stocks with at least a minimum number of days
min_days = 5
stock_day_counts = df_sampled.groupby('STOCK')['DATE'].nunique().reset_index(name='NUM_DAYS')
valid_stocks = stock_day_counts[stock_day_counts['NUM_DAYS'] >= min_days]['STOCK']

# Filter the original dataframe to include only valid stocks
df_valid_stocks = df_sampled[df_sampled['STOCK'].isin(valid_stocks)]

# Calculate mean 10-day volatility
df_volatility_mean = df_valid_stocks.groupby('STOCK')['VOLATILITY_10D'].mean().reset_index()
df_volatility_mean = df_volatility_mean.sort_values(by='VOLATILITY_10D', ascending=False)

# Altair visualizations
selection = alt.selection_single(fields=['STOCK'], empty='none')

# Bar chart for mean 10-day volatility
vol10_chart1 = alt.Chart(df_volatility_mean).mark_bar().encode(
    y=alt.Y('STOCK', sort='-x', title='Stock'),
    x=alt.X('VOLATILITY_10D', title='Mean 10-Day Volatility'),
    tooltip=['STOCK', 'VOLATILITY_10D'],
    color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray'))
).properties(
    title={
        "text": "Mean 10-Day Volatility of Unique Stocks (Descending Order)",
        "fontSize": 20,  # Adjust the font size for this title
        "anchor": "middle"
    },
    width=600,
    height=700
).add_selection(
    selection
)

# Scatter plot with trendline
vol10_chart2 = alt.Chart(df_valid_stocks).mark_point().encode(
    x=alt.X('TWEET_COUNT:Q', title='Tweet Count', scale=alt.Scale(zero=False)),
    y=alt.Y('VOLATILITY_10D:Q', title='10-Day Volatility', scale=alt.Scale(zero=False)),
    tooltip=['DATE', 'TWEET_COUNT', 'VOLATILITY_10D']
).transform_filter(
    selection
).properties(
    title={
        "text": "Correlation between Tweet Count and 10-Day Volatility for Selected Stock",
        "fontSize": 20,  # Adjust the font size for this title
        "anchor": "middle"
    },
    width=800,
    height=700
)

# Add the trendline
trendline = vol10_chart2.transform_regression(
    'TWEET_COUNT', 'VOLATILITY_10D'
).mark_line(color='red')

# Combine the scatterplot and trendline
vol10_chart3 = vol10_chart2 + trendline

# Combine the charts vertically
combined_chart_vol10 = alt.hconcat(
    vol10_chart1,
    vol10_chart3,
).resolve_scale(
    y='independent'
).properties(
    title={
        "text": ["Click A Stock In the Bar Chart to View Its Correlation between 10-Day Volatility and Tweet count"],
        "fontSize": 25,
        "anchor": "middle"
    },
)
# Display the chart
combined_chart_vol10

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

In [12]:
# Calculate mean 30-day volatility
df_volatility_mean_30d = df_valid_stocks.groupby('STOCK')['VOLATILITY_30D'].mean().reset_index()
df_volatility_mean_30d = df_volatility_mean_30d.sort_values(by='VOLATILITY_30D', ascending=False)

# Altair visualizations
selection = alt.selection_single(fields=['STOCK'], empty='none')

# Bar chart for mean 10-day volatility
vol30_chart1 = alt.Chart(df_volatility_mean_30d).mark_bar().encode(
    y=alt.Y('STOCK', sort='-x', title='Stock'),
    x=alt.X('VOLATILITY_30D', title='Mean 30-Day Volatility'),
    tooltip=['STOCK', 'VOLATILITY_30D'],
    color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray'))
).properties(
    title={
        "text": "Mean 30-Day Volatility of Unique Stocks (Descending Order)",
        "fontSize": 20,  # Adjust the font size for this title
        "anchor": "middle"
    },
    width=600,
    height=700
).add_selection(
    selection
)

# Scatter plot with trendline
vol30_chart2 = alt.Chart(df_valid_stocks).mark_point().encode(
    x=alt.X('TWEET_COUNT:Q', title='Tweet Count', scale=alt.Scale(zero=False)),
    y=alt.Y('VOLATILITY_30D:Q', title='30-Day Volatility', scale=alt.Scale(zero=False)),
    tooltip=['DATE', 'TWEET_COUNT', 'VOLATILITY_30D']
).transform_filter(
    selection
).properties(
    title={
        "text": "Correlation between Tweet Count and 30-Day Volatility for Selected Stock",
        "fontSize": 20,  # Adjust the font size for this title
        "anchor": "middle"
    },
    width=800,
    height=700
)

# Add the trendline
trendline = vol30_chart2.transform_regression(
    'TWEET_COUNT', 'VOLATILITY_30D'
).mark_line(color='red')

# Combine the scatterplot and trendline
vol30_chart3 = vol30_chart2 + trendline

# Combine the charts vertically
combined_chart_vol30 = alt.hconcat(
    vol30_chart1,
    vol30_chart3
).resolve_scale(
    y='independent'
).properties(
    title={
        "text": ["Click A Stock In the Bar Chart to View Its Correlation between 30-Day Volatility and Tweet count"],
        "fontSize": 25,
        "anchor": "middle"
    },
)
# Display the chart
combined_chart_vol30

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

In [13]:
finalchart = alt.vconcat(
    combined_chart_vol10,
    combined_chart_vol30
).resolve_scale(
    y='independent'
)

finalchart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

alt.VConcatChart(...)

In [14]:
# Save the chart as a JSON file to the /content directory
finalchart.save('/content/combined_chart.json')

# Download the JSON file
from google.colab import files
files.download('/content/combined_chart.json')

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# Final Version Vol10

# Calculate mean 10-day volatility for each unique stock
df_volatility_mean = df_sampled.groupby('STOCK')['VOLATILITY_10D'].mean().reset_index()

# Sort the stocks by their 10-day volatility in descending order
df_volatility_mean = df_volatility_mean.sort_values(by='VOLATILITY_10D', ascending=False)

# Filter to include only the top 20 stocks with the highest 10-day volatility
df_top20 = df_volatility_mean.head(20)

# Get the list of all unique stocks for the dropdown
all_stocks = sorted(df_sampled['STOCK'].unique().tolist())

all_stocks

[' Amazon',
 '21CF',
 'ASOS',
 'AT&T',
 'Adobe',
 'Amazon',
 'American Express',
 'Apple',
 'Audi',
 'BMW',
 'Bank of America',
 'Bayer',
 'Boeing',
 'Burberry',
 'CBS',
 'Chevron',
 'Cisco',
 'Citigroup',
 'CocaCola',
 'Comcast',
 'Costco',
 'Danone',
 'Disney',
 'Exxon',
 'Facebook',
 'FedEx',
 'Ford',
 'GSK',
 'General Electric',
 'Goldman Sachs',
 'Google',
 'H&M',
 'HP',
 'Home Depot',
 'Honda',
 'Hyundai',
 'IBM',
 'Intel',
 'John Deere',
 "Kellogg's",
 'Kroger',
 "L'Oreal",
 "McDonald's",
 'Microsoft',
 'Morgan Stanley',
 'Nestle',
 'Netflix',
 'Next',
 'Nike',
 'Nissan',
 'Oracle',
 'P&G',
 'PayPal',
 'Pepsi',
 'Reuters',
 'Ryanair',
 'SAP',
 'Samsung',
 'Shell',
 'Siemens',
 'Sony',
 'Starbucks',
 'TMobile',
 'Tesco',
 'Toyota',
 'TripAdvisor',
 'UPS',
 'Verizon',
 'Volkswagen',
 'Walmart',
 'Wells Fargo',
 'Yahoo',
 'adidas',
 'eBay',
 'easyJet',
 'salesforce.com']

In [12]:
# Create a selection dropdown for all stocks
dropdown = alt.binding_select(options=all_stocks, name='Select Stock:')
selection = alt.selection_single(fields=['STOCK'], bind=dropdown, name='Select')

In [13]:
# Create the Bar Chart and Scatter Plot
vol10_chart1 = alt.Chart(df_top20).mark_bar().encode(
    y=alt.Y('STOCK', sort='-x', title='Stock'),
    x=alt.X('VOLATILITY_10D', title='Mean 10-Day Volatility'),
    tooltip=['STOCK', 'VOLATILITY_10D'],
    color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray'))
).properties(
    title='Top 20 Stocks by Mean 10-Day Volatility',
    width=800,
    height=600
).add_selection(
    selection
)

vol10_chart2 = alt.Chart(df_sampled).mark_point().encode(
    x=alt.X('TWEET_COUNT:Q', title='Tweet Count', scale=alt.Scale(zero=False)),
    y=alt.Y('VOLATILITY_10D:Q', title='10-Day Volatility', scale=alt.Scale(zero=False)),
    tooltip=['DATE', 'TWEET_COUNT', 'VOLATILITY_10D']
).transform_filter(
    selection
).properties(
    title='Correlation between Tweet Count and 10-Day Volatility for Selected Stock',
    width=800,
    height=600
)

# Add the trendline
trendline = vol10_chart2.transform_regression(
    'TWEET_COUNT:Q', 'VOLATILITY_10D:Q'
).mark_line(color='red')

# Combine the scatterplot and trendline
vol10_chart3 = vol10_chart2 + trendline

# Combine the charts vertically
combined_chart_vol10 = alt.hconcat(
    vol10_chart1,
    vol10_chart3
).resolve_scale(
    y='independent'
)

# Display the chart
combined_chart_vol10

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)